In [2]:
import pandas as pd
import datetime
import numpy as np
import json


In [3]:
# Parse JSON schema
class FileSchema():

    def json_schema_parser(self, data):
        # read properties from json format schema file
        
        # format_name
        self.format_name = data["format"]["format_name"]

        # CSV params    
        d_csv = {}
        d_csv = data["format"]["csv"]
        
        # set csv params
        self.delimiter = data["format"]["csv"]['delimiter']
        self.compression = data["format"]["csv"]['compression']
        self.header = int(data["format"]["csv"]['header'])

        # set COLUMNS params
        d_cols = data["format"]["columns"]
        self.col_names = []
        self.col_types = {}
        self.col_null = {}
        self.col_keys = {}
        self.col_dt_format = {}
        self.col_dates = []
        self.decimal = '.'
        self.thousand = None
        self.date_parser = None  
     
        # get attributes from each column:
        for i in d_cols:
            self.col_names.append(i)
            for j in d_cols[i]:
                # get column type
                if j == 'type':
                    tipo = d_cols[i][j]
                    if tipo == 'int':
                        self.col_types[i] = 'int64'
                    elif tipo == 'float':
                        self.col_types[i] = 'float64'
                    elif tipo == 'date':
                        self.col_dates.append(i)
                    else: # string
                        self.col_types[i] = 'object'
                
                # column can be null?
                elif j == 'nullable':
                    self.col_null[i] = d_cols[i][j]
                
                # column is key, to avoid duplicate
                elif j == 'key':
                    self.col_keys[i] = d_cols[i][j]
                
                # date format (python format)
                elif j == 'date_format':
                    self.col_dt_format[i] = d_cols[i][j]
                    self.date_parser = d_cols[i][j]
                
                # float decimal char
                elif j == 'decimal':
                    self.decimal = d_cols[i][j]
                # float thousand char
                elif j == 'thousand':
                    self.thousand = d_cols[i][j]

        # print main attributes
        print('format_name = ', self.format_name)
        print('delimiter = ', self.delimiter)
        print('compression = ', self.compression)
        print('header = ', self.header)            
        print('col names: ', self.col_names)
        print('col types: ', self.col_types)
        print('col null: ', self.col_null)
        print('col keys: ', self.col_keys)
        print('col date format: ', self.col_dt_format)
        print('col dates:', self.col_dates)
        print('thousand:', self.thousand)
        print('decimal:', self.decimal)


In [4]:
def valid_col_type(row, col, tipo, fmt, log):
    # Try to validate a column while converting (casting) the value into a certain type.
    # Errors are stored into "log" dictionary
    try:
        #original = str(row[col])
        original = row[col]
        x = ''

        if tipo == 'int':
            ans = int(original)
            
        elif tipo == 'float':
            v_float = original
            if v_float and fmt.thousand:
                v_float = v_float.replace(fmt.thousand, '')
            if v_float and fmt.decimal:
                v_float = v_float.replace(fmt.decimal, '.')
            ans = float(v_float)
            
        elif tipo == 'date':
            ans = datetime.datetime.strptime(original, fmt.date_parser)
           
    except Exception as error:
        v_row = str(row['index'] + 2)
        v_key = 'row:'+  v_row + ',col:' + col + ',TYPE'
        error_str = 'Linha {l} , Coluna {c}, Type : Erro ao tentar converter o valor \'{o}\' para o tipo {t}. | Exception: {e}'.format(
                        l = v_row, o = str(original), t= str(tipo), c = col, e = error)
        log[v_key] = error_str
        ans = pd.np.nan
        
    return ans


In [5]:
def valid_null_col(df, col, index, log):
    # check if the column has null values. If is there any null value, it store the row and column into the log dict.
    nulls = df.loc[df[col].isnull(), 'index'].tolist()
    for i in nulls:
        v_row = i + 2
        v_key = 'row:'+  str(v_row) + ',col:' + col + ',null'
        v_erro = 'Linha {l} , Coluna {c}, NULL : Erro: o valor para esta linha e coluna nao pode ser vazio.'.format(l = v_row, c = col)
        log[v_key] = v_erro

In [7]:
# open the json schema format configuration file
with open('format_sample_001.json') as json_file:
    json_data = json.load(json_file)
    print(json_data)


{'format': {'format_name': 'csv_sample_001', 'csv': {'delimiter': ';', 'compression': 'infer', 'header': '0', 'validate_keys': 'false'}, 'columns': {'CODIGO': {'type': 'int', 'nullable': 'false', 'key': 'true'}, 'NOME': {'type': 'string', 'nullable': 'false', 'size': '150'}, 'CPF': {'type': 'string', 'size': '14', 'nullable': 'true', 'key': 'false'}, 'NASCIMENTO': {'type': 'date', 'date_format': '%d/%m/%Y', 'min_date': '01/01/1900', 'max_date': '31/12/2050', 'nullable': 'true', 'key': 'false'}, 'SALDO': {'type': 'float', 'nullable': 'yes', 'key': 'false', 'decimal': ',', 'thousand': '.', 'negative': 'true', 'currency_symbol': 'R$'}}}}


In [8]:
# parse the schema into the fmt instance
fmt = FileSchema()
fmt.json_schema_parser(json_data)

format_name =  csv_sample_001
delimiter =  ;
compression =  infer
header =  0
col names:  ['CODIGO', 'NOME', 'CPF', 'NASCIMENTO', 'SALDO']
col types:  {'CODIGO': 'int64', 'NOME': 'object', 'CPF': 'object', 'SALDO': 'float64'}
col null:  {'CODIGO': 'false', 'NOME': 'false', 'CPF': 'true', 'NASCIMENTO': 'true', 'SALDO': 'yes'}
col keys:  {'CODIGO': 'true', 'CPF': 'false', 'NASCIMENTO': 'false', 'SALDO': 'false'}
col date format:  {'NASCIMENTO': '%d/%m/%Y'}
col dates: ['NASCIMENTO']
thousand: .
decimal: ,


In [9]:
# read the sample file into a dataframe
p_csv = 'sample_file_01.csv'
df1 = pd.read_csv(p_csv, delimiter = fmt.delimiter , encoding='UTF-8')
# add a index to the df
df1 = df1.reset_index()

In [10]:
df1.head(15)

,index,CODIGO,NOME,CPF,NASCIMENTO,SALDO
0,0,1001,Tina Voldaren,100.100.100-10,15/11/1980,459
1,1,1002,Ana Fier,200.200.200-20,01/01/2000,"500,00"
2,2,1003,Samantha Smallcock,300.200.100-30,31/07/1977,"3.333,30"
3,3,1004,Jennifer Ravena (Jinni),400.300.200-10,04/04/1944,NaN
4,4,1005,Barbara Patrick,100.100.100-11,25/12/1999,241.17
5,5,1006,Selena Herrick (Sena),200.200.200-21,15/11/1980,"20.177,87"
6,6,1007,Sarah Varney (Sadie),300.200.100-31,01/01/2000,"114,57"
7,7,x,x,x,x,x
8,8,1008,Vera Cassidy,400.300.200-11,31/07/1977,"51,27"
9,9,100.9,Genésia Aparecida,100.100.100-12,04/04/1944,"-12,03"


In [11]:
df1.dtypes

index          int64
CODIGO        object
NOME          object
CPF           object
NASCIMENTO    object
SALDO         object
dtype: object

In [12]:
flog = {}
# validating the column types for CODIGO (int), SALDO (float) and NASCIMENTO (date)
df1['CODIGO'] = df1.apply(valid_col_type, axis=1, col='CODIGO', tipo='int', fmt = fmt, log = flog)
df1['SALDO'] = df1.apply(valid_col_type, axis=1, col='SALDO', tipo='float', fmt = fmt, log = flog)
df1['NASCIMENTO'] = df1.apply(valid_col_type, axis=1, col='NASCIMENTO', tipo='date', fmt = fmt, log = flog)

In [14]:
df1.dtypes

index                  int64
CODIGO               float64
NOME                  object
CPF                   object
NASCIMENTO    datetime64[ns]
SALDO                float64
dtype: object

In [15]:
# show rows with null CODIGO
dfn = df1.loc[df1['CODIGO'].isnull(), :]
dfn.head()


,index,CODIGO,NOME,CPF,NASCIMENTO,SALDO
7,7,NaN,x,x,NaT,NaN
9,9,NaN,Genésia Aparecida,100.100.100-12,1944-04-04,-12.03
13,13,NaN,NaN,NaN,NaT,NaN


In [16]:
ncols = fmt.col_null
print(ncols)

# validate columns that cannot be null "nullable = false"
for col in ncols:
    if ncols[col] == 'false':
        valid_null_col(df1, col, 'index', flog)


{'CODIGO': 'false', 'NOME': 'false', 'CPF': 'true', 'NASCIMENTO': 'true', 'SALDO': 'yes'}


In [17]:
flog

{'row:9,col:CODIGO,TYPE': "Linha 9 , Coluna CODIGO, Type : Erro ao tentar converter o valor 'x' para o tipo int. | Exception: invalid literal for int() with base 10: 'x'",
 'row:11,col:CODIGO,TYPE': "Linha 11 , Coluna CODIGO, Type : Erro ao tentar converter o valor '100.9' para o tipo int. | Exception: invalid literal for int() with base 10: '100.9'",
 'row:15,col:CODIGO,TYPE': "Linha 15 , Coluna CODIGO, Type : Erro ao tentar converter o valor 'nan' para o tipo int. | Exception: cannot convert float NaN to integer",
 'row:5,col:SALDO,TYPE': "Linha 5 , Coluna SALDO, Type : Erro ao tentar converter o valor 'nan' para o tipo float. | Exception: 'float' object has no attribute 'replace'",
 'row:9,col:SALDO,TYPE': "Linha 9 , Coluna SALDO, Type : Erro ao tentar converter o valor 'x' para o tipo float. | Exception: could not convert string to float: 'x'",
 'row:15,col:SALDO,TYPE': "Linha 15 , Coluna SALDO, Type : Erro ao tentar converter o valor 'nan' para o tipo float. | Exception: 'float' o

In [18]:
# save errors into log file, with same input file name, but adding the extension ".err"
import csv
f = open(p_csv + ".err", "w")
w = csv.writer(f, lineterminator = '\n')
for key, val in flog.items():
    w.writerow([val])
f.close()